# Query Registry Users

In [27]:
# set lifemonitor root path
lifemonitor_root = "/mnt/data/projects/@crs4/EOSC-Life/Repositories/life_monitor_rachk8s"
%cd -q {lifemonitor_root}

In [28]:
import requests
from examples import utils # just a simple utility to share code between netbooks

In [29]:
# LifeMonitor URLs
lm_base_url = "https://localhost:8000"

In [30]:
token = utils.fetch_registry_token('seek')
utils.pp(token)

{ 'access_token': 'f0PNUHwUpFfNVF64nwBiR4vZvZGRxIbA7AYawtfOrN',
  'expires_in': 864000,
  'scope': 'read write',
  'token_type': 'Bearer'}


In [33]:
# HTTP settings to connect to LifeMonitor
s = requests.session() # actually not mandatory, but just to share some settings among requests

# if you use self-signed certificates,
# you have to uncomment the line below to disable SSL verification
s.verify = False

# Update headers with the OAuth2 token
s.headers.update({'Authorization': f"Bearer {token['access_token']}"})

In [34]:
# Get registry users
response = s.get(f"{lm_base_url}/users")
assert response.status_code == 200, f"Unexpected error {response.status_code}: {response.content}"
registry_users = response.json()
utils.pp(registry_users)

{ 'items': [ { 'id': 7,
               'identity': { 'provider': { 'name': 'seek',
                                           'type': 'seek',
                                           'uri': 'https://seek:3000',
                                           'userinfo_endpoint': 'https://seek:3000/people/current?format=json'},
                             'sub': '1'},
               'username': '0vFhLRgbA2'}]}


In [35]:
# redirect user to the registration process
utils.pp(f"{lm_base_url}/oauth2/login/seek") # <- 'seek' is the registry name

# ... and login as 'user1' (password: 'workflowhub')

'https://localhost:8000/oauth2/login/seek'


In [37]:
# Check registry users
response = s.get(f"{lm_base_url}/users")
assert response.status_code == 200, f"Unexpected error {response.status_code}: {response.content}"
registry_users = response.json()
utils.pp(registry_users)

{ 'items': [ { 'id': 7,
               'identity': { 'provider': { 'name': 'seek',
                                           'type': 'seek',
                                           'uri': 'https://seek:3000',
                                           'userinfo_endpoint': 'https://seek:3000/people/current?format=json'},
                             'sub': '1'},
               'username': '0vFhLRgbA2'},
             { 'id': 8,
               'identity': { 'provider': { 'name': 'seek',
                                           'type': 'seek',
                                           'uri': 'https://seek:3000',
                                           'userinfo_endpoint': 'https://seek:3000/people/current?format=json'},
                             'sub': '3'},
               'username': 'gmqmftYqoS'}]}


In [38]:
# Check registry users
response = s.get(f"{lm_base_url}/users/{registry_users['items'][0]['id']}")
assert response.status_code == 200, f"Unexpected error {response.status_code}: {response.content}"
registry_users = response.json()
utils.pp(registry_users)

{ 'id': 7,
  'identity': { 'provider': { 'name': 'seek',
                              'type': 'seek',
                              'uri': 'https://seek:3000',
                              'userinfo_endpoint': 'https://seek:3000/people/current?format=json'},
                'sub': '1'},
  'username': '0vFhLRgbA2'}
